# graph.GraphComponentFactory

> TODO fill in description

In [ ]:
#| default_exp graph.graph_component

In [ ]:
#| hide
from nbdev.showdoc import *;

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#|export
from __future__ import annotations
import asyncio
from abc import ABC, abstractmethod
import typing
from types import MappingProxyType
from typing import Type, Optional, Callable, Any, Union, Tuple, Coroutine, List, Dict
from enum import Enum
import inspect
import uuid
from datetime import datetime, timezone
from inspect import signature
from fastcore.basics import patch_to

import fbdev
from fbdev._utils import AttrContainer, TaskManager, SingletonMeta, StateCollection, StateHandler
from fbdev.comp.packet import Packet
from fbdev.comp.port import PortType, PortSpec, PortSpecCollection, Port, PortCollection
from fbdev.comp.base_component import BaseComponent
from fbdev.graph.graph_spec import GraphSpec, NodeSpec, EdgeSpec
from fbdev.graph.net import Edge, Node, NodePort, Net
from fbdev.exceptions import NodeError, EdgeError

In [ ]:
#|hide
show_doc(fbdev.graph.graph_component.GraphComponentFactory)

---

### GraphComponentFactory

>      GraphComponentFactory ()

*Helper class that provides a standard way to create an ABC using
inheritance.*

In [ ]:
#|export
class GraphComponentFactory(BaseComponent):
    is_factory = True
    expose_graph = True
    
    graph: GraphSpec = None
    
    def __init__(self):
        super().__init__()
        self._parent_net: Net = None # Is set by Net in Net.start()
        self._nodes: Dict[str, Node] = {}
        self._edges: Dict[str, Edge] = {}
            
    @property
    def nodes(self) -> MappingProxyType[str, Node]: return MappingProxyType(self._nodes)
    @property
    def edges(self) -> MappingProxyType[str, Edge]: return MappingProxyType(self._edges)
    
    def _handle_node_exception(self, task:asyncio.Task, exception:Exception, source_trace:Tuple):
        try: raise NodeError() from exception
        except NodeError as e: self._task_manager.submit_exception(task, e, source_trace)
    
    def _handle_edge_exception(self, task:asyncio.Task, exception:Exception, source_trace:Tuple):
        try: raise EdgeError() from exception
        except EdgeError as e: self._task_manager.submit_exception(task, e, source_trace)
    
    @classmethod
    def create_component(cls, graph, expose_graph=True) -> Type[BaseComponent]:
        graph = graph.copy()
        graph.make_readonly()
        return cls._create_component_class(class_attrs={
            'graph' : graph,
            'expose_graph' : expose_graph,
            'port_specs' : graph._port_specs
        })
        
    async def _post_start(self):
        for node_spec in self.graph.nodes.values():
            self._nodes[node_spec.id] = Node(node_spec, self._parent_net)
            self._nodes[node_spec.id]._task_manager.subscribe(self._handle_node_exception)
        for edge_spec in self.graph.edges.values():
            self._edges[edge_spec.id] = Edge(edge_spec, self._parent_net)
            self._edges[edge_spec.id]._task_manager.subscribe(self._handle_edge_exception)
        
        for node in self._nodes.values():
            await node.start()
        for edge in self.edges.values():
            edge.start()
            
    async def _pre_terminate(self):
        for node in self._nodes.values():
            await node.terminate()
        for edge in self.edges.values():
            await edge.terminate()

In [ ]:
class BaseFooComponent(BaseComponent):
    @abstractmethod
    async def main(self): ...
    async def _post_start(self): self._task_manager.create_task(self.main())

class FooComponent1(BaseFooComponent):
    port_specs = PortSpecCollection(
        PortSpec(PortType.INPUT, "inp"),
        PortSpec(PortType.OUTPUT, "out"),
    )
    async def main(self):
        packet = await self.ports.input.inp.get()
        print(await packet.consume())
        await self.ports.output.out.put(Packet('there'))
        
class FooComponent2(BaseFooComponent):
    port_specs = PortSpecCollection(
        PortSpec(PortType.INPUT, "inp"),
        PortSpec(PortType.OUTPUT, "out"),
    )
    async def main(self):
        packet = await self.ports.input.inp.get()
        print(await packet.consume())
        await self.ports.output.out.put(Packet('world'))
        
graph = GraphSpec(PortSpecCollection())

graph.add_graph_port(PortSpec(PortType.INPUT, "inp"))
graph.add_graph_port(PortSpec(PortType.OUTPUT, "out"))

node1 = graph.add_node(FooComponent1)
node2 = graph.add_node(FooComponent2)

graph.ports.input.inp >> node1.ports.input.inp
node1 >> node2
node2.ports.output.out >> graph.ports.output.out

graph.display_mermaid(hide_unconnected_ports=True)

```mermaid
flowchart 
    subgraph FooComponent1["FooComponent1[]"]
        FooComponent1__C__input__D__inp[inp]
        FooComponent1__C__output__D__out[out]
    end
    subgraph FooComponent2["FooComponent2[]"]
        FooComponent2__C__input__D__inp[inp]
        FooComponent2__C__output__D__out[out]
    end
    GRAPH__C__input__D__inp[inp]
    GRAPH__C__output__D__out[out]
    FooComponent1__C__output__D__out --> FooComponent2__C__input__D__inp
    GRAPH__C__input__D__inp -.-> FooComponent1__C__input__D__inp
    FooComponent2__C__output__D__out -.-> GRAPH__C__output__D__out
    classDef input fill:#13543e;
    classDef output fill:#0d1b59;
    classDef subgraph_zone fill:#000;
    class FooComponent1__C__input__D__inp,FooComponent2__C__input__D__inp,GRAPH__C__input__D__inp input;
    class FooComponent1__C__output__D__out,FooComponent2__C__output__D__out,GRAPH__C__output__D__out output;
```

In [ ]:
graph_component = fbdev.graph.GraphComponentFactory.create_component(graph)
net_spec = NodeSpec(graph_component)
net = Net(net_spec)

packet = Packet('hello')

async def get_output():
    packet = await net.component_process.get_packet((PortType.OUTPUT, 'out'))
    print(await packet.consume())

await net.start()
await net.exec_coros(
    net.component_process.put_packet((PortType.INPUT, 'inp'), packet),
    get_output(),
)

hello
there
world
